$\textbf{Nombre:}$ Pedro Ramos Suárez.

Utilizaremos las siguientes bibliotecas:

    · Numpy: Para cálculos numéricos y arrays
    · Sympy: Para obtener fórmulas y derivadas.
    · Pandas: Para salidas en forma de tabla.
    · Matplotlib.pyplot: Para representar gráficas.
    · IPython.display: Para imprimir ecuaciones por pantalla.

In [ ]:
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Opción para que imprima todas las columnas (aunque no entren en una tabla)
pd.set_option('display.max_rows', None)

# Aumentamos la precisión en las tablas
pd.set_option("display.precision", 12)

# Ejercicio 1

#### Obtenga mediante la fórmula de interpolación en el espacio $\mathbb{P}_{2}$ una fórmula para aproximar f''(a) del tipo combinación de $f(a-h), f(a)$ y $f(a+h)$.

Nuestros nodos son $a-h, a$ y $a+h$. Los definimos como símbolos de sympy:

In [ ]:
f = sp.Function('f')
a,h = sp.symbols('a, h')

Con esto, nuestros nodos y sus imágenes son:

In [ ]:
x = [a-h, a, a+h]
y = [f(x[0]), f(x[1]), f(x[2])]

Construimos el polinomio de interpolación:

In [ ]:
z = sp.Symbol('z')
p0 = y[0]
p1 = p0 + (z-x[0])/(x[1]-x[0])*(y[1]-y[0])
D = sp.Symbol('D')
p2 = p1 + (z-x[0])*(z-x[1])*D

Resolvemos la ecuación y despejamos los valores de $D$ para obtener el polinomio:

In [ ]:
D = sp.solve(p2.subs({z:x[2]})-y[2],D)[0]
p2 = p1 + (z-x[0])*(z-x[1])*D
display(p2)

La aproximación para la primera derivada es:

In [ ]:
sp.diff(p2,z).subs({z:a}).simplify()

La aproximación para la segunda derivada es:

In [ ]:
sp.diff(p2,z,2)

# Ejercicio 2

#### Con la fórmula obtenida en el ejercicio 1, halle una tabla de aproximaciones y errores de $f_{1}''(2.5)$, siendo $f_{1}(x) = x^{x}$, para $h = 10^{-i}, i = 1, ..., 5$.

Definimos la función $f_{1}$ e implementamos el método obtenido en el ejercicio anterior:

In [ ]:
def f1(x):
    return x**x

def interpolatorioSegundaDerivada(a, h, f):
    return (-2*f(a) + f(a-h) + f(a+h))/(h**2)

Calculamos las aproximaciones para $h = 10^{-i}, i = 1,...,5$:

In [ ]:
def ejercicio2():
    iteraciones = 5
    a = 2.5
    
    # Calculamos el valor exacto usando sympy
    x = sp.symbols('x')
    dx = sp.diff(x**x)
    ddx = sp.diff(dx)
    
    valor = ddx.subs(x, a)
    
    # Imprimimos por pantalla
    print('La seguna derivada de f es: ')
    display(ddx)
    print('y su valor en {} es {}.\n'.format(a, valor))
    
    # Calculamos las aproximaciones y los errores
    aproximaciones = [interpolatorioSegundaDerivada(a, 10**(-i), f1) for i in range(1,iteraciones+1)]
    errores = [np.abs(aproximaciones[i] - valor) for i in range(iteraciones)]
    
    # Utilizamos un dataFrame de pandas para la salida en forma de tabla
    data = np.array([['h', 'Aproximación', 'Error'],
                   ['0.1', aproximaciones[0], errores[0]],
                   ['0.01', aproximaciones[1], errores[1]],
                   ['0.001', aproximaciones[2], errores[2]],
                   ['0.0001', aproximaciones[3], errores[3]],
                   ['0.00001', aproximaciones[4], errores[4]]])
    
    print(pd.DataFrame(data=data[1:,1:],index=data[1:,0],columns=data[0,1:]))
    
ejercicio2()

# Ejercicio 3

#### Sea $f_2(x)=\frac{x^2+40}{x+\sqrt{5x}+7}$. Calcule una tabla que recoja las derivadas de $f_2$ en $x_i=1,2,\ldots,10$, utilizando alguna de las fórmulas de derivación numérica de primer orden obtenidas al inicio de la práctica, con $h=10^{-3}$, y muestre al mismo tiempo el error cometido en cada punto. Repita el ejercicio con la fórmula centrada obtenida para la derivada primera y, finalmente, para la obtenida en el ejercicio 1 (con respecto a la segunda derivada).

Calculamos la fórmula de interpolación con nodos $a, a+h, a+2h$:

In [ ]:
f = sp.Function('f')
a,h = sp.symbols('a, h')
x = [a, a+h, a+2*h]
y = [f(x[0]), f(x[1]), f(x[2])]
z = sp.Symbol('z')
p0 = y[0]
p1 = p0 + (z-x[0])/(x[1]-x[0])*(y[1]-y[0])
D = sp.Symbol('D')
p2 = p1 + (z-x[0])*(z-x[1])*D
D = sp.solve(p2.subs({z:x[2]})-y[2],D)[0]
p2 = p1 + (z-x[0])*(z-x[1])*D
sp.diff(p2,z).subs({z:a}).simplify()

Definimos la función $f_{2}$ e implementamos la fórmula anterior:

In [ ]:
def f2(x):
    return (x**2 + 40)/(x + np.sqrt(5*x) + 7)

def interpolatorioDerivadaNoCentrada(a, h, f):
    return (-3*f(a) + 4*f(a+h) - f(a+2*h))/(2*h)

def interpolatorioDerivadaCentrada(a, h, f):
    return (-f(a-h) + f(a+h))/(2*h)

Con todo esto, ya podemos calcular la aproximación de la primera derivada con nodos centrados (ejercicio 1) y con los nodos no centrados (que acabamos de obtener), y de la segunda derivada con nodos centrados (ejercicio 1):

In [ ]:
def ejercicio3():
    # Calculamos el valor exacto usando sympy para el error
    x = sp.symbols('x')
    dx = sp.diff((x**2+40)/(x+sp.sqrt(5*x)+7))
    
    # Imprimimos la función:
    print(' # Función: ')
    display((x**2+40)/(x+sp.sqrt(5*x)+7))
    
    # Calculamos las derivadas
    noCentradas = [interpolatorioDerivadaNoCentrada(i, 10**(-3), f2) for i in range(1,11)]
    centradas = [interpolatorioDerivadaCentrada(i, 10**(-3), f2) for i in range(1,11)]
    
    # Calculamos los errores
    errorNoCentradas = [np.abs(noCentradas[i-1] - dx.subs(x, i).evalf()) for i in range(1,11)]
    errorCentradas = [np.abs(centradas[i-1] - dx.subs(x, i).evalf()) for i in range(1,11)]
    
    # Utilizamos un dataFrame de pandas para la salida en forma de tabla
    data = np.array([['a', 'Valor', 'No Centrada', 'Error No Centrada','Centrada', 'Error Centrada'],
                   ['1',  dx.subs(x, 1).evalf(),  noCentradas[0], errorNoCentradas[0], centradas[0], errorCentradas[0]],
                   ['2',  dx.subs(x, 2).evalf(),  noCentradas[1], errorNoCentradas[1], centradas[1], errorCentradas[1]],
                   ['3',  dx.subs(x, 3).evalf(),  noCentradas[2], errorNoCentradas[2], centradas[2], errorCentradas[2]],
                   ['4',  dx.subs(x, 4).evalf(),  noCentradas[3], errorNoCentradas[3], centradas[3], errorCentradas[3]],
                   ['5',  dx.subs(x, 5).evalf(),  noCentradas[4], errorNoCentradas[4], centradas[4], errorCentradas[4]],
                   ['6',  dx.subs(x, 6).evalf(),  noCentradas[5], errorNoCentradas[5], centradas[5], errorCentradas[5]],
                   ['7',  dx.subs(x, 7).evalf(),  noCentradas[6], errorNoCentradas[6], centradas[6], errorCentradas[6]],
                   ['8',  dx.subs(x, 8).evalf(),  noCentradas[7], errorNoCentradas[7], centradas[7], errorCentradas[7]],
                   ['9',  dx.subs(x, 9).evalf(),  noCentradas[8], errorNoCentradas[8], centradas[8], errorCentradas[8]],
                   ['10', dx.subs(x, 10).evalf(), noCentradas[9], errorNoCentradas[9], centradas[9], errorCentradas[9]]])
    
    print(' # Primera derivada: ')
    display(dx)
    print(pd.DataFrame(data=data[1:,1:],index=data[1:,0],columns=data[0,1:]))
    
    # Calculamos la segunda derivada con la fórmula del ejercicio 1
    ddx = sp.diff(dx)
    segundasDerivadas = [interpolatorioSegundaDerivada(i, 10**(-3), f2) for i in range(1,11)]
    errorSegundasDerivadas = [np.abs(segundasDerivadas[i-1] - ddx.subs(x, i).evalf()) for i in range(1,11)]
    
    # Utilizamos un dataFrame de pandas para la salida en forma de tabla
    data = np.array([['a', 'Valor', 'Aproximación', 'Error'],
                   ['1',  ddx.subs(x, 1).evalf(),  segundasDerivadas[0], errorSegundasDerivadas[0]],
                   ['2',  ddx.subs(x, 2).evalf(),  segundasDerivadas[1], errorSegundasDerivadas[1]],
                   ['3',  ddx.subs(x, 3).evalf(),  segundasDerivadas[2], errorSegundasDerivadas[2]],
                   ['4',  ddx.subs(x, 4).evalf(),  segundasDerivadas[3], errorSegundasDerivadas[3]],
                   ['5',  ddx.subs(x, 5).evalf(),  segundasDerivadas[4], errorSegundasDerivadas[4]],
                   ['6',  ddx.subs(x, 6).evalf(),  segundasDerivadas[5], errorSegundasDerivadas[5]],
                   ['7',  ddx.subs(x, 7).evalf(),  segundasDerivadas[6], errorSegundasDerivadas[6]],
                   ['8',  ddx.subs(x, 8).evalf(),  segundasDerivadas[7], errorSegundasDerivadas[7]],
                   ['9',  ddx.subs(x, 9).evalf(),  segundasDerivadas[8], errorSegundasDerivadas[8]],
                   ['10', ddx.subs(x, 10).evalf(), segundasDerivadas[9], errorSegundasDerivadas[9]]])
    
    print('\n # Segunda derivada: ')
    display(ddx)
    print(pd.DataFrame(data=data[1:,1:],index=data[1:,0],columns=data[0,1:]))
    
ejercicio3()

# Ejercicio 4

#### Divida el intervalo $[1,2]$ en 100 partes iguales y aplique las fórmulas del rectángulo, Simpson y trapecio compuestas para aproximar la integral en dicho intervalo de $f_1$. Compare dichos resultados.

Primero definimos métodos para representar gráficamente los métodos por pantalla:

In [ ]:
def linea(z,xL,yL,xR,yR):
    """Línea uniendo los puntos (xL,yL) y (xR,yR)"""
    return (z-xL)/(xR-xL)*yR+(xR-z)/(xR-xL)*yL

def parabola(z,xL,yL,xM,yM,xR,yR):
    """Parábola uniendo los puntos (xL,yL), (xM,yM) y (xR,yR)"""
    D1 = (yM-yL)/(xM-xL)
    D2 = (yR-yL-(xR-xL)*D1)/((xR-xL)*(xR-xM))
    return yL+D1*(z-xL)+D2*(z-xL)*(z-xM)

Definimos todos los métodos (rectángulo izquierdo, rectángulo derecho, Simpson y trapecio compuestas) y la opción para representarlas:

In [ ]:
def rectIzq(a, b, n, f, plot):
    h = (b - a) / n
    
    if plot:
        nx = n
        nxx = 10 * n
        
        xx = np.linspace(float(a), float(b), nxx)
        yy = f(xx)
        
        x = np.linspace(float(a), float(b), nx)
        y = f(x)

        fig,ax = plt.subplots(figsize=(8,8))
        ax.plot(x[:-1], y[:-1], 'ro', markersize = 0.5)
        ax.plot(xx, yy,'k',label = 'Función del integrando')
        
        for k in range(1,nx):
            zz = np.linspace(float(x[k-1]), float(x[k]), nxx)
            yk = [linea(z, x[k-1], y[k-1], x[k], y[k-1]) for z in zz]
            ax.fill_between(zz, 0, yk);

        ax.legend(loc=9)
        ax.set_xlabel('$x$', fontsize=18)
        ax.set_ylabel('$y$', fontsize=18)
        ax.set_title('Fórmula compuesta de los rectángulos a la izquierda');
        
    return h * sum([f(a + i * h) for i in range(0, n)])

def rectDer(a, b, n, f, plot):
    h = (b - a) / n
    
    if plot:
        nx = n
        nxx = 10 * n
        
        xx = np.linspace(float(a), float(b), nxx)
        yy = f(xx)
        
        x = np.linspace(float(a), float(b), nx)
        y = f(x)
        
        fig,ax = plt.subplots(figsize=(8,8))
        ax.plot(x[1:], y[1:], 'ro', markersize = 0.5)
        ax.plot(xx, yy, 'k', label = 'Función del integrando')
        for k in range(1,nx):
            zz = np.linspace(float(x[k-1]), float(x[k]),nxx)
            yk = [linea(z, x[k-1], y[k], x[k], y[k]) for z in zz]
            ax.fill_between(zz, 0, yk);

        ax.legend(loc=9)
        ax.set_xlabel('$x$',fontsize=18)
        ax.set_ylabel('$y$',fontsize=18)
        ax.set_title('Fórmula compuesta de los rectángulos a la derecha');
        
    return h * sum([f(a + (i + 1) * h) for i in range(0, n)])
    
def puntoMedio(a, b, n, f, plot):
    h = (b - a) / n
    
    if plot:
        nx = n
        nxx = 10 * n
        
        xx = np.linspace(float(a), float(b), nxx)
        yy = f(xx)
        
        x = np.linspace(float(a), float(b), nx)
        y = f(x)
        
        x0 = (x[1:] + x[:-1]) / 2
        y0 = f(x0)

        fig,ax = plt.subplots(figsize=(8,8))
        ax.plot(x0, y0, 'ro', markersize = 0.5)
        ax.plot(xx, yy, 'k',label = 'Función del integrando')
        for k in range(1,nx):
            zz = np.linspace(float(x[k-1]), float(x[k]), nxx)
            yk = [linea(z, x[k-1], (y[k-1]+y[k])/2, x[k], (y[k-1]+y[k])/2) for z in zz]
            ax.fill_between(zz,0,yk);

        ax.legend(loc=9)
        ax.set_xlabel('$x$',fontsize=18)
        ax.set_ylabel('$y$',fontsize=18)
        ax.set_title('Fórmula compuesta de los rectángulos del punto medio');
        
    return h * sum([f(a + (i + 1/2) * h) for i in range(0, n)])
    
def trapecio(a, b, n, f, plot):
    h = (b - a) / n
    
    if plot:
        nx = n
        nxx = 10 * n
        
        xx = np.linspace(float(a), float(b), nxx)
        yy = f(xx)
        
        x = np.linspace(float(a), float(b), nx)
        y = f(x)

        fig,ax = plt.subplots(figsize=(8,8))
        ax.plot(x, y, 'ro', markersize = 0.5)
        ax.plot(xx, yy, 'k', label = 'Función del integrando')
        for k in range(1,nx):
            zz = np.linspace(float(x[k-1]), float(x[k]), nxx)
            yk = [linea(z, x[k-1], y[k-1], x[k], y[k]) for z in zz]
            ax.fill_between(zz, 0, yk);

        ax.legend(loc=9)
        ax.set_xlabel('$x$',fontsize=18)
        ax.set_ylabel('$y$',fontsize=18)
        ax.set_title('Fórmula del trapecio compuesta');
    
    return h / 2 * (f(a) + 2 * sum([f(a + i * h) for i in range(1, n)]) + f(b))
    
def simpson(a, b, n, f, plot):
    h = (b - a) / n
    sum1 = sum([f(a + i * h) for i in range(2, n, 2)])
    sum2 = sum([f(a + i * h) for i in range(1, n, 2)])
    
    if plot:
        nx = 2 * n
        nxx = 10 * nx
        
        xx = np.linspace(float(a), float(b), nxx + 1)
        yy = f(xx)
        
        x = np.linspace(float(a), float(b), nx + 1)
        y = f(x)

        fig,ax = plt.subplots(figsize=(8,8))
        ax.plot(x, y, 'ro', markersize = 0.5)
        ax.plot(xx, yy, 'k', label = 'Función del integrando')

        for k in range(1, n+1):
            zz = np.linspace(float(x[2*(k-1)]), float(x[2*k]), nxx)
            yk = [parabola(z, x[2*(k-1)], y[2*(k-1)], x[2*k-1], y[2*k-1], x[2*k], y[2*k]) for z in zz]
            ax.fill_between(zz, 0, yk);

        ax.legend(loc=9)
        ax.set_xlabel('$x$', fontsize=18)
        ax.set_ylabel('$y$', fontsize=18)
        ax.set_title('Fórmula de Simpson compuesta');
    
    return h / 3 * (f(a) + 2 * sum1 + 4 * sum2 + f(b))

Con todo esto, ya podemos obtener los valores de cada método y sus errores. Este ejercicio lo definimos de forma más general ya que también lo vamos a usar en el siguiente ejercicio:

In [ ]:
def ejercicio4(f, a, b, integral, n, plot = False):
    # Calculamos los valores
    valorRectIzq = rectIzq(a, b, n, f, plot)
    valorRectDer = rectDer(a, b, n, f, plot)
    valorPuntoMedio = puntoMedio(a, b, n, f, plot)
    valorTrapecio = trapecio(a, b, n, f, plot)
    valorSimpson = simpson(a, b, n, f, plot)
        
    # Utilizamos un dataFrame de pandas para la salida en forma de tabla
    data = np.array([['', 'Valor', 'Error'],
                   ['Rectángulo izquierda',  valorRectIzq, np.abs(valorRectIzq - integral.evalf())],
                   ['Rectángulo derecha',  valorRectDer, np.abs(valorRectDer - integral.evalf())],
                   ['Punto medio',  valorPuntoMedio, np.abs(valorPuntoMedio - integral.evalf())],
                   ['Trapecio',  valorTrapecio, np.abs(valorTrapecio - integral.evalf())],
                   ['Simpson',  valorSimpson, np.abs(valorSimpson - integral.evalf())]])
    
    print(pd.DataFrame(data=data[1:,1:],index=data[1:,0],columns=data[0,1:]))
    
a = 1
b = 2
x = sp.symbols('x', positive=True)
integral = sp.Integral(x**x, (x, a, b))
ejercicio4(f1, a, b, integral, 100, True)

# Ejercicio 5

#### Repita el ejercicio 4 para $f_2$. 

Utilizamos todos las funciones definidas en el ejercicio anterior con la función $f_{2}$:

In [ ]:
a = 1
b = 2
x = sp.symbols('x', positive=True)
integral = sp.Integral((x**2+40)/(x+sp.sqrt(5*x)+7), (x, a, b))
ejercicio4(f2, a, b, integral, 100, True)

# Ejercicio 6

#### Sea $f_3(x)=x^{15} e^x$ en $[0,2]$. Vamos a dividir el intervalo en $10\times 2^n$ subintervalos, es decir, $10,\,20,\,40,\, 80,\ldots $ y a aplicar la fórmula de Simpson compuesta hasta que la diferencia entre dos aproximaciones consecutivas (por ejemplo, podrían ser con $20$ y $40$ subintervalos) sea menor que $10^{-2}$, dando en tal caso por buena la última aproximación obtenida. Programe y calcule dicha aproximación. Compare ambas aproximaciones con el valor exacto.

Definimos la nueva función $f_{3}$:

In [ ]:
def f3(x):
    return x**15 * np.exp(x)

Calculamos los valores de todas las iteraciones hasta que el error sea menor que $10^{-2}$ y sus errores:

In [ ]:
def ejercicio6():
    i = 0
    
    a = 0
    b = 2
    
    x = sp.symbols('x', positive=True)
    integral = sp.Integral(x**15 * sp.exp(x), (x, a, b))
    
    continuar = True
    value = np.inf
    
    data = np.array(['Puntos', 'Valor', 'Error', 'Diferencia con iteración anterior'])
    
    while continuar:
        oldValue = value
        n = 10 * 2 ** i
        value = simpson(a, b, n, f3, False)
        
        # Si la diferencia entre dos iteraciones es menor que 0.01, hemos terminado
        if np.abs(value - oldValue) < 10**(-2):
            continuar = False
            
        i += 1
        
        data = np.vstack((data, np.array([n, value, np.abs(value - integral.evalf()), np.abs(value - oldValue)])))
          
    print(pd.DataFrame(data=data[1:,1:],index=data[1:,0],columns=data[0,1:]))
        
ejercicio6()

# Ejercicio 7

#### Calcule las fórmulas gaussianas con $2$ y $3$ nodos,en el intervalo $[-1,1]$, siendo la función peso el valor absoluto de la variable. Aplíquelas para aproximar la función $x\; e^x$ en $[-1,1]$ y compare los resultados con el valor exacto (organizando los cálculos de forma adecuada).

Definimos la nueva función $f_{4}$ y el método de la gaussiana:

In [ ]:
def f4(x):
    return x * sp.exp(x)

def gaussiana(n, a, b, f):
    grexact = 2*n-1
    
    p = sp.symbols('p1:'+ str(n+1)) # Generamos una tupla de n nodos
    nodos1 = list(p)
    
    c = sp.symbols('c1:'+ str(n+1)) # Generamos una tupla de n coeficientes
    coefs1 = list(c)
    
    incogs1 = coefs1 + nodos1
    
    ecs1 = [np.dot([(z**i).subs({z:nodos1[j]}) for j in range(n)],coefs1)-sp.Rational(b**(i+1)-a**(i+1),(i+1)) for i in range(grexact+1)]
    solsGauss1 = sp.solve(ecs1,incogs1)
    
    for i in range(n):  # Y así le asignaríamos a los coeficientes las soluciones obtenidas 
        coefs1[i] = solsGauss1[0][i]
    for i in range(n):  # Y así le asignaríamos a los nodos las soluciones obtenidas
        nodos1[i] = solsGauss1[0][n+i]
        
    # Aquí realizamos un simple producto escalar multiplicando término a término los valores de los coeficientes por
    # la evaluación de la función evaluada en los nodos obtenidos y sumando posteriormente esos términos obtenidos.    
    formGauss1 = np.dot([f(nodos1[i]) for i in range(n)],coefs1)
        
    return(formGauss1)

Calculamos los valores obtenidos con el método de la gaussiana con 2 y 3 nodos:

In [ ]:
def ejercicio7():
    a = -1
    b = 1
    
    # Calculamos el valor real de la integral:
    valor = sp.integrate(x * sp.exp(x), (x, -1, 1))
    
    # Con 2 nodos:
    nodos2 = gaussiana(2, a, b, f)
    nodos2f = gaussiana(2, a, b, f4)
    
    print('Con 2 nodos:')
    display(nodos2)
    print('En el caso de:')
    display(f4(x))
    print('tenemos:')
    display(nodos2f)
    print('y obtenemos de valor: {}, y el error es: {}.'.format(nodos2f.evalf(), np.abs(nodos2f.evalf() - valor.evalf())))
    
    print('\n -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- \n')
    
    # Con 3 nodos:
    nodos3 = gaussiana(3, a, b, f)
    nodos3f = gaussiana(3, a, b, f4)
    
    print('Con 3 nodos:')
    display(nodos3)
    print('En el caso de:')
    display(f4(x))
    print('tenemos:')
    display(nodos3f)
    print('y obtenemos de valor: {}, y el error es: {}.'.format(nodos3f.evalf(), np.abs(nodos3f.evalf() - valor.evalf())))
    
ejercicio7()

# Ejercicio 8

#### Programar las técnicas de integración de Romberg y adaptativa, para después aplicarlas a la aproximación de la siguiente integral $$\int_a^b p(x)\, dx$$ siendo  $\;a=\displaystyle\min_{0\leq i\leq 7}{d_i}$, $\;b=\displaystyle\max_{0\leq i\leq 7}{d_i}$ y  $$p(x)=d_0 + d_1 x + d_2 x^2 + d_3 x^3+ d_4 x^4 + d_5 x^5 + d_6 x^6 + d_7 x^7 $$ (siendo $d_0, d_1, \ldots, d_7$ los dígitos de su DNI, pasaporte o tarjeta de residente).

Como mi DNI es 76591270M, tenemos:
$$p(x) = 7 + 6x + 5x^{2} + 9x^{3} + 1x^{4} + 2x^{5} + 7x^{6} + 0x^{7} = 7 + 6x + 5x^{2} + 9x^{3} + x^{4} + 2x^{5} + 7x^{6}$$
por lo que $a = 0$ y $b = 9$.

Definimos los métodos de integración de Romberg y la integral adaptativa:

In [ ]:
# DNI: 76591270M
# a = 0, b = 9

def p(x):
    return 7 + 6 * x + 5 * x ** 2 + 9 * x ** 3 + 1 * x ** 4 + 2 * x ** 5 + 7 * x ** 6 + 0 * x ** 7

def romberg(a, b, n, f):
    # Matriz con los valores obtenidos con el método de Romberg (sólo el triángulo inferior)
    R = np.zeros((n, n))
    
    for j in range(n):
        for k in range(j+1):
            # Los valores R[j,0] los cálculamos directamente del método del trapecio
            if k == 0:
                R[j,k] = trapecio(a, b, 2**j, f, False)
            # Los demás valores los calculamos a partir de iteraciones anteriores
            else:
                num = 4 ** k * R[j, k-1] - R[j-1, k-1]
                den = 4 ** k - 1
                R[j,k] = num/den
            
    return(R[n-1, n-1])

# Definimos S(a, b) usada en la integración adaptativa
def s(a, b, f):
    h = (b - a) / 2
    m = (a + b) / 2
    return h / 3 * (f(a) + 4 * f(m) + f(b))
    
def adaptativa(a, b, f, epsilon, i):
    h = (b - a) / 2
    m = (a + b) / 2
    
    # Valores de las dos integrales
    valor1 = s(a, m, f)
    valor2 = s(m, b, f)
    
    # Si el error es mayor que epsilon, dividimos el intervalo y repetimos el algoritmo
    if not np.abs(s(a,b,f) - valor1 - valor2) < 10 * epsilon:
        valor1 = adaptativa(a, m, f, epsilon/2, i+1)
        valor2 = adaptativa(m, b, f, epsilon/2, i+1)
        
    return valor1 + valor2

Y los aplicamos a nuestra función:

In [ ]:
def ejercicio8():
    a = 0
    b = 9
    
    # Calculamos el valor real de la integral
    x = sp.symbols('x')
    valor = sp.integrate(7 + 6 * x + 5 * x ** 2 + 9 * x ** 3 + 1 * x ** 4 + 2 * x ** 5 + 7 * x ** 6, (x, a, b))
    
    # Calculamos los valores obtenidos por los métodos
    valorRomberg = romberg(a, b, 10, p)
    valorAdaptativa = adaptativa(a, b, p, 10**(-5), 0)
    
    # Imprimimos los resultados por pantalla
    print('Valor real de la integral: {}.'.format(valor.evalf()))
    print('Con la integración de Romberg obtenemos: {}, y de error: {}.'.format(valorRomberg, np.abs(valorRomberg - valor)))
    print('Col la integral adaptativa obtenemos: {}, y de error: {}.'.format(valorAdaptativa, np.abs(valorAdaptativa - valor)))
    
ejercicio8()